### AIR Project

## Imports and specific settings

In [52]:
import ast
import string
import sys
from collections import OrderedDict

import torch
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Word2Vec
import multiprocessing
import time
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torchmetrics.classification import BinaryF1Score

In [53]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print("running models with {}".format(device))

running models with cpu


## Loading of dataset and nan-removal

In [54]:
def pre_process():
    data = pd.read_csv('WELFake_Dataset.csv', index_col=0)
    print(data.shape)
    # display(data[:300])
    for i,x in data.iterrows():
        if len(str(x["text"])) <= 10:
            data.loc[i, "text"] = np.nan
        if len(str(x["title"])) <= 10:
            data.loc[i, "title"] = np.nan

    data.dropna(inplace=True)
    print(data.shape)
    data.reset_index(drop=True, inplace=True)
    data.to_csv("data/data.csv")
    display(data[:300])

## Tokenization

In [55]:
def tokenize():
    stop = stopwords.words('english')
    stemmer = SnowballStemmer('english')
    punc = [u'\u201c',u'\u201d',u'\u2018',u'\u2019',u'\u2024',u'\u2025',u'\u2026',u'\u2027']
    # print(punc)
    data = pd.read_csv('data/data.csv', index_col=0)
    titles = list()
    texts = list()
    for i, row in data.iterrows():
        title = str(row["title"])
        text = str(row["text"])
        t1 = ""
        for c in title:
            if not (c in string.punctuation or c in punc):
                t1 += c
            else:
                t1 += " "
        t2 = ""
        for c in text:
            if not (c in string.punctuation or c in punc):
                t2 += c
            else:
                t2 += " "
        title_tokens = nltk.tokenize.word_tokenize(t1)
        text_tokens = nltk.tokenize.word_tokenize(t2)
        # title_filtered = [w.lower() for w in title_tokens if not w.lower() in string.punctuation]
        # title_filtered = [w.lower() for w in title_filtered if not w.lower() in punc]
        title_filtered = [w.lower() for w in title_tokens if not w.lower() in stop]
        title_stemmed = [stemmer.stem(w) for w in title_filtered]
        # text_filtered = [w.lower() for w in text_tokens if not w.lower() in string.punctuation]
        # text_filtered = [w.lower() for w in text_filtered if not w.lower() in punc]
        text_filtered = [w.lower() for w in text_tokens if not w.lower() in stop]
        text_stemmed = [stemmer.stem(w) for w in text_filtered]
        # print(title_stemmed)
        # print(text_stemmed)
        titles.append(title_stemmed)
        texts.append(text_stemmed)
        if i % 5000 == 0:
            print(i)
    d = {"title":titles, "text":texts, "label":data["label"]}
    data_cleaned = pd.DataFrame(data=d)
    # data_cleaned["title"] = titles
    # data_cleaned["text"] = texts
    data_cleaned.to_csv("data/data_token.csv")
# tokenize()

## Creating and Training Word2Vec Model

In [56]:
load_model_from_disc = True
w2v_model = None
data = pd.read_csv('data_tokenized/data_token.csv', index_col=0)
# for i, row in data.iterrows():
#     print(type(row["title"]))
#     print(row)
#     data.loc[i, "title"] = ast.literal_eval(row["title"])
#     data.loc[i, "text"] = ast.literal_eval(row["text"])
if load_model_from_disc:
    try:
        w2v_model = Word2Vec.load("word2vec.model")
    except:
        pass

if w2v_model is None or not load_model_from_disc:
    if load_model_from_disc:
        print("Could not load model from disc. Training model...")
    else:
        print("Loading from disc deactivated. Training model...")

    class MySentences(object):
        def __init__(self, data):
            self.data = data

        def __iter__(self):
            for doc in pd.concat([data["text"], data["title"]]): #change to "title" or combine both
                doc = ast.literal_eval(doc)
                yield doc

    sentences = MySentences(data)

    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=20,
                         window=2,
                         sample=6e-5,
                         alpha=0.03,
                         min_alpha=0.0007,
                         negative=20,
                         workers=cores-1)

    w2v_model.build_vocab(sentences, progress_per=10000)
    t = time.time()
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))
    w2v_model.save("word2vec.model")
else:
    print("Model loaded from disc.")

Model loaded from disc.


In [57]:
# calculate similarity
w2v_model.wv.similarity("amazon", 'nazi')

-0.12460326

In [58]:
# calculate similarity
w2v_model.wv.similarity("obama", 'trump')

0.5865986

In [59]:
# find out which element doesn't match
w2v_model.wv.doesnt_match(['amazon', 'obama', 'trump'])

'amazon'

In [60]:
# Which word is to obama as georg is to bush?
w2v_model.wv.most_similar(positive=["obama", "georg"], negative=["bush"], topn=3)

[('barack', 0.6315594911575317),
 ('presid', 0.49833399057388306),
 ('outgo', 0.46703284978866577)]

In [61]:
# e.g. words most similar to obama
w2v_model.wv.most_similar(positive=["obama"])

[('barack', 0.83445143699646),
 ('presid', 0.6639242172241211),
 ('predecessor', 0.6574732661247253),
 ('administr', 0.6533365845680237),
 ('outgo', 0.6066954135894775),
 ('trump', 0.5865985155105591),
 ('bachelet', 0.5682237148284912),
 ('bush', 0.5610581040382385),
 ('undo', 0.5586986541748047),
 ('obama\x92', 0.5423570275306702)]

In [62]:
# e.g. words most similar to obama
w2v_model.wv.most_similar(positive=["presid"])

[('barack', 0.6808820962905884),
 ('obama', 0.6639242172241211),
 ('45th', 0.6278534531593323),
 ('successor', 0.6069088578224182),
 ('administr', 0.6057536005973816),
 ('predecessor', 0.6040663719177246),
 ('donald', 0.5951759815216064),
 ('outgo', 0.5931932330131531),
 ('trump', 0.5852926969528198),
 ('presidenti', 0.5783258676528931)]

## Creating Doc2Vec
word2vec for each word with average over document

In [63]:
# creates w2v representation for all documents and titles
def doc2vec():
    titles = list()
    texts = list()
    start = time.time()
    for i, row in data.iterrows():
        vec_title = np.zeros(shape=w2v_model.vector_size)
        vec_text = np.zeros(shape=w2v_model.vector_size)
        tit = ast.literal_eval(row["title"])
        tex = ast.literal_eval(row["text"])
        tit_cnt = 0
        tex_cnt = 0
        for word in tit:
            try:
                vec_title += w2v_model.wv[word]
            except KeyError:
                # print("Didn't find word {}".format(word))
                tit_cnt += 1
                pass
        for word in tex:
            try:
                vec_text += w2v_model.wv[word]
            except KeyError:
                # print("Didn't find word {}".format(word))
                tex_cnt += 1
                pass
        if len(tit) > tit_cnt:
            vec_title /= (len(tit) - tit_cnt)
        if len(tex) > tex_cnt:
            vec_text /= (len(tex) - tex_cnt)
        titles.append(vec_title.tolist())
        texts.append(vec_text.tolist())
        if i % 5000 == 0:
            print("[{}/{}] - {:.1f}s".format(i, len(data.index), time.time() - start))
    end = time.time()
    print("creating doc2vec took {:.1f}s".format(end - start))
    d = {"title":titles, "text":texts, "label":data["label"]}
    data_w2v = pd.DataFrame(data=d)
    data_w2v.to_csv("data/data_w2v.csv")
    display(data_w2v[:100])
# doc2vec()

## Train-Test-split and Dataloader Creation

In [64]:
def collate_batch(batch):
    labels = list()
    texts = list()
    for (_text, _label) in batch:
        labels.append(_label)
        texts.append(_text)

    return torch.tensor(texts), torch.tensor(labels)

In [65]:
batch_size = 100
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0,
          'collate_fn': collate_batch}


data_d2v = pd.read_csv("data/data_w2v.csv", index_col=0)
titles = list()
texts = list()
# print("interpreting data")
# for i, row in data_d2v.iterrows():
#     titles.append(ast.literal_eval(row["title"]))
#     texts.append(ast.literal_eval(row["text"]))
# print("done interpreting data")
# data_d2v["title"] = titles
# data_d2v["text"] = texts

data_d2v_title = data_d2v[["title", "label"]].copy()
data_d2v_text = data_d2v[["text", "label"]].copy()
X_train_title, X_test_title, y_train_title, y_test_title = train_test_split(data_d2v_title["title"], data_d2v_title["label"], test_size=0.15, random_state=42, shuffle=True)
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(data_d2v_text["text"], data_d2v_text["label"], test_size=0.15, random_state=42, shuffle=True)

X_train_title.reset_index(drop=True, inplace=True)
X_test_title.reset_index(drop=True, inplace=True)
y_train_title.reset_index(drop=True, inplace=True)
y_test_title.reset_index(drop=True, inplace=True)
X_train_text.reset_index(drop=True, inplace=True)
X_test_text.reset_index(drop=True, inplace=True)
y_train_text.reset_index(drop=True, inplace=True)
y_test_text.reset_index(drop=True, inplace=True)


class data_set(Dataset):
    def __init__(self, X, y):
        super(Dataset, self).__init__()
        assert len(X.index) == len(y.index)
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X.index)

    def __getitem__(self, index):
        return ast.literal_eval(self.X[index]), self.y[index]

train_dataset_title = data_set(X_train_title, y_train_title)
test_dataset_title = data_set(X_test_title, y_test_title)
train_dataset_text = data_set(X_train_text, y_train_text)
test_dataset_text = data_set(X_test_text, y_test_text)

train_dataloader_title = DataLoader(train_dataset_title, **params)
test_dataloader_title = DataLoader(test_dataset_title, **params)
train_dataloader_text = DataLoader(train_dataset_text, **params)
test_dataloader_text = DataLoader(test_dataset_text, **params)

for batch, (X, y) in enumerate(train_dataloader_title):
    # print(X)
    print(X.shape)
    print(y.shape)
    break


torch.Size([100, 100])
torch.Size([100])


## Helper Functions

In [66]:
def add_metrics_to_log(log, metrics, y_true, y_pred, prefix=''):
    for metric in metrics:
        q = metric(y_true, y_pred)
        log[prefix + metric.__name__] = q
    return

def log_to_message(log, precision=4):
    fmt = "{0}: {1:." + str(precision) + "f}"
    return "    ".join(fmt.format(k, v) for k, v in log.items())

class ProgressBar(object):
    """Cheers @ajratner"""

    def __init__(self, n, length=40):
        # Protect against division by zero
        self.n      = max(1, n)
        self.nf     = float(n)
        self.length = length
        # Precalculate the i values that should trigger a write operation
        self.ticks = set([round(i/100.0 * n) for i in range(101)])
        self.ticks.add(n-1)
        self.bar(0)

    def bar(self, i, message=""):
        """Assumes i ranges through [0, n-1]"""
        if i in self.ticks:
            b = int(np.ceil(((i+1) / self.nf) * self.length))
            sys.stdout.write("\r[{0}{1}] {2}%\t{3}".format(
                "="*b, " "*(self.length-b), int(100*((i+1) / self.nf)), message
            ))
            sys.stdout.flush()

    def close(self, message=""):
        # Move the bar to 100% before closing
        self.bar(self.n-1)
        sys.stdout.write("\n{0}\n\n".format(message))
        sys.stdout.flush()

## Training Loop

In [67]:

epochs = 100

def train(dataloader, model, loss_fn, optimizer):
    start_time = time.time()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    total_loss = 0
    f1 = 0
    correct = 0
    f1_score = BinaryF1Score().to(device)
    pb = ProgressBar(size/batch_size)
    log = OrderedDict()
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred.squeeze(), y.float())
        total_loss += loss.item()
        f1 += f1_score(pred.squeeze(), y)
        correct += (pred.squeeze().int() == y).float().sum()
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        log['loss'] = float(loss) / (batch + 1)
        log['f1'] = float(f1) / (batch + 1)
        log['accuracy'] = correct / ((batch + 1) * batch_size)
        log['time'] = time.time() - start_time
        pb.bar(batch, log_to_message(log))
    pb.close(log_to_message(log))
    return total_loss, (f1/num_batches).item(), (correct/(num_batches*batch_size)).item(), time.time()-start_time

## Model for testing of training loop

In [68]:
class SimpleLinearModel(nn.Module):
    def __init__(self):
        super(RelevanceNet, self).__init__()
        self.fc1 = nn.Linear(100, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, vec):
        x = self.relu(self.fc1(vec))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        return torch.sigmoid(self.fc4(x))

model = SimpleLinearModel().to(device)

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

train(train_dataloader_title, model, loss_fn, optimizer)

[========================================] 100%		loss: 0.0008    f1: 0.8384    accuracy: 0.4950    time: 36.2576
loss: 0.0008    f1: 0.8384    accuracy: 0.4950    time: 36.2576



(214.91504491865635,
 0.8384043574333191,
 0.49498337507247925,
 36.261555671691895)